# Orange County House Search

The following are some considerations for buying a house in Orange County, CA

In [33]:
#hide
# export to: _notebooks/2022-01-02-HouseSearch2021.ipynb
#!pip install --upgrade -q --no-deps pandas
!pip install --upgrade -q plotly
!pip install -q itables
#!pip install -q plotlyhtmlexporter

from google.colab import data_table
from IPython.display import display, Markdown, Latex
from itables import interactive, show
import pandas as pd
import warnings
import plotly
import plotly.graph_objects as go
import plotly.express as px
print(f'pandas version: {pd.__version__}')
print(f'plotly version: {plotly.__version__}')

#pd.set_option('display.width', 2000)
pd.set_option('display.max_colwidth', None)
#pd.options.plotting.backend = "plotly"

data_table.enable_dataframe_formatter()
warnings.filterwarnings('ignore') # get rid of annoying warnings so they don't appear in notebook outputs

pandas version: 1.1.5
plotly version: 5.5.0


In [34]:
#hide
sheetId = '1az-gajkCWX4GwgJnNMFX8gfeRvhRxAY3QMTowj0YTwE'
hdf = pd.read_csv(f'https://docs.google.com/spreadsheets/d/{sheetId}/export?format=csv',
                 skiprows=2)
hdf = hdf.iloc[:,1:] # skip first blank column
hdf.rename(columns={col:col.replace(' ','').lower() for col in hdf.columns},inplace=True)
for col in hdf.columns[4:]:
  if hdf[col].dtype=='O':
    hdf[col] = pd.to_numeric(hdf[col].str.replace('[^0-9.]','',regex=True))
hdf.delta_price = hdf.delta_price/100
hdf['home2'] = hdf.home.apply(lambda x: x.split(',')[0])
hdf['zip'] = hdf['home'].str.split(',').str[-1].str.split(' ').str[-1]
hdf.head()

Loading... (need help?)


In [35]:
#hide
# Read in HPI data and convert to Pandas DataFrame
sheetId = '1az-gajkCWX4GwgJnNMFX8gfeRvhRxAY3QMTowj0YTwE'
hpidf =  pd.read_csv(f'https://docs.google.com/spreadsheets/d/{sheetId}/export?gid=1182115701&format=csv',
                 skiprows=6,dtype='str')
hpidf = pd.DataFrame(hpidf.values[1:],columns=['zip','year','percent_change','hpi','hpi_1990','hpi_2000'])
for col in hpidf.columns[1:]:
  hpidf[col] = pd.to_numeric(hpidf[col].str.replace('[^0-9.]','',regex=True),errors='coerce')
hpidf.head()

Loading... (need help?)


In [36]:
#hide
azdf = hdf.groupby(['area','zip']).count().reset_index()[['area','zip']]

In [37]:
#hide
hpiyeardf = azdf.merge(hpidf,left_on='zip',right_on='zip',how='inner')
hpiyeardf = hpiyeardf[hpiyeardf.year>2000].groupby(['area','year']).mean()['hpi_2000']#.reset_index()
hpiyeardf.head()

Loading... (need help?)


In [38]:
#hide
p21df = hdf.groupby('area').mean()['delta_price'].reset_index()
p21df = hpiyeardf.reset_index()[hpiyeardf.reset_index().year==2019].merge(p21df,left_on='area',right_on='area')
p21df['year']=2021
p21df.hpi_2000 = p21df.hpi_2000*(1+p21df.delta_price)
p21df = p21df[['area','year','hpi_2000']]

In [39]:
#hide
# add projection of 2021 based on selling prices of homes (this is an assumption as these houses may be overpriced and may not sell)
hpiyeardf = pd.concat([hpiyeardf,p21df.groupby(['area','year']).mean()['hpi_2000']])

## Houses of interest

The following houses are sorted by the highest average rating, followed by the price:

In [40]:
#hide_input
# Filter only For Sale with high ratings
hoidf = hdf.loc[(hdf.status=="For Sale")&(hdf.aveq>=7),["status","home","zillowlink","price","sqft","aveq"]] 
#hoidf = hoidf.style.format({'zillowlink': lambda x: f'<a href="{x}">{x}</a>'})
#hoidf['zillowlink2'] = hoidf.zillowlink.apply(lambda x : f'<a href="{x}>{x}</a>')  #f'<a href="{hoidf.zillowlink}>{hoidf.zillowlink}</a>'
#hoidf['zillowlink2'] = hoidf.zillowlink.apply(lambda x : f'["{x}]({x})')  #f'<a href="{hoidf.zillowlink}>{hoidf.zillowlink}</a>'
#show(hoidf,paging=False,scrollY="800px")
hoidf.style.format({'zillowlink': lambda x: f'<a href="{x}">{x}</a>'}).hide_index()
#data_table.DataTable(hoidf,include_index=False,num_rows_per_page=20)

status,home,zillowlink,price,sqft,aveq
For Sale,"20 Winslow St, Ladera Ranch, CA 92694",https://www.zillow.com/homedetails/20-Winslow-St-Ladera-Ranch-CA-92694/51669389_zpid/,1700000,3433,9.000000
For Sale,"27641 Deputy Cir, Laguna Hills, CA 92653","https://www.zillow.com/homes/27641-Deputy-Cir,-Laguna-Hills,-CA-92653-_rb/25551152_zpid/",3995000,6742,9.200000
For Sale,"12 Vista Montemar, Laguna Niguel, CA 92677",https://www.zillow.com/homedetails/12-Vista-Montemar-Laguna-Niguel-CA-92677/25572417_zpid/,3888000,5300,8.400000
For Sale,"21861 Montbury Dr, Lake Forest, CA 92630",https://www.zillow.com/homes/21861-Montbury-Drive_rb/25528526_zpid/,1798000,3458,8.000000
For Sale,"23 Via Lucena, San Clemente, CA 92673",https://www.zillow.com/homedetails/23-Via-Lucena-San-Clemente-CA-92673/69250433_zpid/,2400000,3786,8.000000
For Sale,"26936 Falling Leaf Dr, Laguna Hills, CA 92653","https://www.zillow.com/homes/26936-Falling-Leaf-Dr-Laguna-Hills,-CA-92653_rb/25548202_zpid/",1599000,3370,7.000000


## Ratings Needed

The following people still need to rate the following (For Sale/Coming Soon) homes:

In [41]:
#hide_input
raters = [c for c in hdf.columns if c[-1]=='q' and c!='aveq']
ntrdf = hdf.query(f"status=='For Sale' or status =='Coming Soon'").loc[:,['home2','zillowlink']+raters].melt(id_vars = ['home2','zillowlink'],value_vars=raters).query("value=='NaN'").assign(name = lambda x: x.variable.str[:-1].str.capitalize()).loc[:,['name','home2','zillowlink']] 
ntrdf.style.format({'zillowlink': lambda x: f'<a href="{x}">{x}</a>'}).hide_index()
#show(ntrdf,scrollY="800px",paging=False)#.groupby('variable').home2.apply(lambda x: '<br>'.join(x) ).reset_index()



name,home2,zillowlink
Kate,20 Winslow St,https://www.zillow.com/homedetails/20-Winslow-St-Ladera-Ranch-CA-92694/51669389_zpid/
Kate,21861 Montbury Dr,https://www.zillow.com/homes/21861-Montbury-Drive_rb/25528526_zpid/
Kate,23 Via Lucena,https://www.zillow.com/homedetails/23-Via-Lucena-San-Clemente-CA-92673/69250433_zpid/
Kate,63 Tessera Ave,"https://www.zillow.com/homes/63-Tessera-Ave-Foothill-Ranch,-CA-92610_rb/25523926_zpid/"
Kate,25551 El Capitan Ln,https://www.zillow.com/homedetails/25551-El-Capitan-Ln-Laguna-Hills-CA-92653/25545436_zpid/?
Kate,26936 Falling Leaf Dr,"https://www.zillow.com/homes/26936-Falling-Leaf-Dr-Laguna-Hills,-CA-92653_rb/25548202_zpid/"
Kate,26722 Cuenca Dr,https://www.zillow.com/homes/26722-Cuenca-Drive_rb/25603347_zpid/
Jay,20 Winslow St,https://www.zillow.com/homedetails/20-Winslow-St-Ladera-Ranch-CA-92694/51669389_zpid/
Jay,21861 Montbury Dr,https://www.zillow.com/homes/21861-Montbury-Drive_rb/25528526_zpid/
Jay,23 Via Lucena,https://www.zillow.com/homedetails/23-Via-Lucena-San-Clemente-CA-92673/69250433_zpid/


## Plots to explore houses

### Plot: Rating vs Price
A comparison of the rating vs asking price of houses are shown below, along with their current status and area. Click the corresponding unit in the legend to remove houses from the plot.  

In [42]:
#hide_input
# Rating vs. Price
filter = hdf.aveq>5 #suppress houses with lower than 5 rating
#colorProp = "area"
colorProp = "status"
hovertemplate = "<b>%{hovertext}</b><br>(%{customdata[7]})<br><br>%{customdata[0]}<br>$%{x:,} (%{customdata[3]:.0%})<br>%{customdata[5]} bed, %{customdata[6]} bath<br>%{customdata[1]} sqft<br>%{customdata[2]:,} lot size<br>$%{customdata[4]} HOA<br>%{y}<extra></extra> average rating"
symbol_map = {"For Sale":"circle",
              "Coming Soon":"circle",
              "Off Market":"square",
              "Pending":"cross",
              "Under Contract":"cross",
              "Sold":"x"}

fig = px.scatter(hdf[filter],x="price",y="aveq",text="home2",size='sqft',color="area",symbol="status",symbol_map=symbol_map,hover_name="home2",hover_data=["area","sqft","lot","price","delta_price","hoafee","bed","bath","status"])
trendline = px.scatter(hdf[filter],x="price",y="aveq",trendline="ols",trendline_options=dict(log_x=True)).data[1]
fig.update_traces(hovertemplate=hovertemplate)
fig.add_traces(trendline)
fig.update_yaxes(title_text="Rating (average)")
fig.update_traces(textposition='top center',textfont_size=10)#,marker=dict(line=dict(width=hdf[filter].delta_price*10,color='DarkSlateGrey'))) #<-- this doesn't work!
fig.update_layout(title="Orange County Houses: Rating vs Price")
fig.add_annotation(xref="paper",yref="paper",x=0.5,y=0,showarrow=False,text="note: point size = sqft")
fig.write_html('H21_rating_vs_price.html')
fig.show()

### Plot: Price Change since 2019
The median increase in asking price relative to 2019 is `40%`. Houses below this median are depicted in the green area of the plot, while houses above this are depicted in the red area of the plot.

In [43]:
#hide_input
#Rating vs. Price change
symbol_map = {"For Sale":"circle",
              "Coming Soon":"circle",
              "Off Market":"square",
              "Pending":"cross",
              "Under Contract":"cross",
              "Sold":"x"}

trend = px.scatter(hdf,y="aveq",x="delta_price",trendline="ols") # the trendline option will add a second trace (trend.data[1]) that we can add to the scatter plot we want
fig = px.scatter(hdf,y="aveq",x="delta_price",color="area", size="sqft", symbol="status",symbol_map=symbol_map,text="home2",hover_name="home2",hover_data=["area","sqft","lot","price","delta_price","hoafee","bed","bath","status"])
hovertemplate = "<b>%{hovertext}</b><br>(%{customdata[7]})<br><br>%{customdata[0]}<br>$%{customdata[3]:,} (%{x:.0%})<br>%{customdata[5]} bed, %{customdata[6]} bath<br>%{customdata[1]} sqft<br>%{customdata[2]:,} lot size<br>$%{customdata[4]} HOA<br>%{y}<extra></extra> average rating"
fig.update_traces(hovertemplate=hovertemplate)
fig.update_traces(textposition='top center',textfont_size=8)
fig.add_trace(trend.data[1])
fig.update_layout(title="Orange County Houses: Rating vs Price change (since 2019)")
fig.update_xaxes(title_text="Price change (since 2019)",tickformat='.0%')
fig.update_yaxes(title_text="Rating (average)")
fig.add_vrect(x0=-0.1, x1=hdf.delta_price.mean(), line_width=0, fillcolor="green", opacity=0.1)
fig.add_vrect(x0=hdf.delta_price.mean(), x1=1.05, line_width=0, fillcolor="red", opacity=0.1)
fig.add_annotation(xref="paper",yref="paper",x=0.5,y=0,showarrow=False,text="note: green/red boundary is at the median price increase")
fig.write_html('H21_rating_vs_delta_price.html')
fig.show()

### What are our ratings sensitive to? Rating, Price and Square Footage

Our average house rating grows *less than linearly* with the increase in square footage. 

* There are several houses with an `average rating > 7` beyond `3500` square feet. 

In [44]:
#hide_input
# Rating vs Sqft

trend = px.scatter(hdf,y="aveq",x="sqft",trendline="ols",trendline_options=dict(log_x=True)).data[1] # the trendline option will add a second trace (trend.data[1]) that we can add to the scatter plot we want
fig = px.scatter(hdf,y="aveq",x="sqft",color="area",hover_name="home2",hover_data=["area","sqft","lot","price","delta_price","hoafee","bed","bath","status"])
hovertemplate = "<b>%{hovertext}</b><br>(%{customdata[7]})<br><br>%{customdata[0]}<br>$%{customdata[2]:,} (%{customdata[3]:.0%})<br>%{customdata[5]} bed, %{customdata[6]} bath<br>%{x} sqft<br>%{customdata[1]:,} lot size<br>$%{customdata[4]} HOA<br>%{y}<extra></extra> average rating"
fig.update_traces(hovertemplate=hovertemplate)
fig.add_trace(trend)
fig.update_layout(title="Orange County Houses: Rating vs Square Footage")
fig.update_xaxes(title_text="Square Feet")
fig.update_yaxes(title_text="Rating (average)")
fig.write_html('H21_rating_vs_sqft.html')
fig.show()

The following shows that we are ***mostly insensitive to price after the square footage of the house is accounted for*** (with `0.3` change in average rating per `100` increase in price/sqft):

In [45]:
#hide_input
# Rating vs Price per sqft

trend = px.scatter(hdf,y="aveq",x="psqft",trendline="ols").data[1] # the trendline option will add a second trace (trend.data[1]) that we can add to the scatter plot we want
fig = px.scatter(hdf,y="aveq",x="psqft",color="area",hover_name="home2",hover_data=["area","sqft","lot","price","delta_price","hoafee","bed","bath","status"])
hovertemplate = "<b>%{hovertext}</b><br>(%{customdata[8]})<br><br>%{customdata[0]}<br>$%{customdata[3]:,} (%{customdata[4]:.0%})<br>%{customdata[6]} bed, %{customdata[7]} bath<br>%{customdata[1]} sqft<br>%{customdata[2]:,} lot size<br>$%{customdata[5]} HOA<br>%{y}<extra></extra> average rating"
fig.update_traces(hovertemplate=hovertemplate)
fig.add_trace(trend)
fig.update_layout(title="Orange County Houses: Rating vs Price per square foot")
fig.update_xaxes(title_text="Price per Square Foot")
fig.update_yaxes(title_text="Rating (average)")
fig.write_html('H21_rating_vs_psqft.html')
fig.show()

In [46]:
#hide_input
# Sqft vs Price

trend = px.scatter(hdf,y="sqft",x="price",trendline="ols").data[1] # the trendline option will add a second trace (trend.data[1]) that we can add to the scatter plot we want
fig = px.scatter(hdf,y="sqft",x="price",color="area",hover_name="home2",hover_data=["area","sqft","lot","price","delta_price","hoafee","bed","bath","aveq","status"])
hovertemplate = "<b>%{hovertext}</b><br>(%{customdata[7]})<br><br>%{customdata[0]}<br>$%{x:,} (%{customdata[2]:.0%})<br>%{customdata[4]} bed, %{customdata[5]} bath<br>%{y} sqft<br>%{customdata[1]:,} lot size<br>$%{customdata[3]} HOA<br>%{customdata[6]}<extra></extra> average rating"
fig.update_traces(hovertemplate=hovertemplate)
fig.add_trace(trend)
fig.update_layout(title="Orange County Houses: Sqft vs Price")
fig.update_xaxes(title_text="Price")
fig.update_yaxes(title_text="Square Feet")
fig.write_html('H21_sqft_vs_price.html')
fig.show()

In [47]:
#hide
# Mount google drive (so I can try to export whole notebook as html)

# from google.colab import drive
# drive.mount('/content/drive/')

In [48]:
#hide
# Convert the notebook to html

# %%shell
# cp '/content/drive/MyDrive/Colab Notebooks/HouseSearch2021.ipynb' ./
# jupyter nbconvert --to html /content/HouseSearch2021.ipynb

## Historical Prices in the OC Areas

Below are shown the historical prices of houses in each area relative to their estimated price in  2000. The final points in 2021 are based on average asking price per area of the houses we have noted. 

### Methodology
Historical prices are taken from the [Federal Housing Finance Agency's House Price Index (HPI 2000)](https://www.fhfa.gov/DataTools/Downloads/Pages/House-Price-Index.aspx). This gives the measure $hpi\_2000(X)$ for any year $X$ in any zip code. The 2021 points are calculated as follows: 

hpi_2000(2021) $\equiv$ hpi_2000(2019) * ( 1 + $\langle\frac{zillow\_asking(house_i)}{zillow\_2019(house_i)}\rangle_{i \in area})$

In [49]:
#hide_input
# Make Historical house prices figure
trendline = px.scatter(hpiyeardf.reset_index().groupby('year').mean()['hpi_2000'].reset_index(),x='year',y='hpi_2000',trendline='ols').data[1].update(line_color='black')
#fig = hpiyeardf.unstack(level=0).plot();
fig = px.scatter(hpiyeardf.reset_index(),x='year',y='hpi_2000',color='area').update_traces(mode='lines')

#fig = hpiyeardf.unstack(level=0).plot();
fig.add_traces(trendline)
fig.update_yaxes(title_text="HPI (% of 2000 baseline)")
fig.update_layout(title="Orange County Houses: Historical Prices since 2000")
fig.add_annotation(xref="paper",yref="paper",x=0.5,y=0,showarrow=False,text="note: 2021 values are projected from selling prices over 2019 price")


The sharp rise in 2021 is confirmed by [a recent FHFA report for 2021 Q3](https://www.fhfa.gov/Videos/Pages/FHFA-House-Price-Index-2021-Q3.aspx).

## Historical Data on houses sold from OCRE 

The Orange County Real Estate Inc provides [access to some data](https://www.ocrealestateinc.com/search/market_report_search/), about 2k homes sold in the last 3 years in the specific regions of OC that we have been looking at. 

In [50]:
#hide
# Read in ocre data
sheetId = '1az-gajkCWX4GwgJnNMFX8gfeRvhRxAY3QMTowj0YTwE'
ocredf =  pd.read_csv(f'https://docs.google.com/spreadsheets/d/{sheetId}/export?gid=853689105&format=csv',parse_dates=True)#,dtype='str')
ocre_descdf = ocredf[['home','zip','area','desc']]
ocredf.drop(columns=['Unnamed: 0','desc'],inplace=True)
ocredf.closing_date = pd.to_datetime(ocredf.closing_date)
ocredf['price_sold_list_ratio'] = ocredf.price_sold / ocredf.price_list
ocredf['year_sold'] = ocredf.closing_date.dt.year
#ocredf.head()

In [51]:
#hide_input
filter = 'price_list < 2e6 and price_list > 7.5e5 and closing_date > 20190701'
bin_freq = '3M'
focredf = ocredf.query(filter)

trendline = px.scatter(focredf.groupby([pd.Grouper(key='closing_date',freq=bin_freq)]).mean().price_sold.reset_index(),x='closing_date',y='price_sold',trendline="lowess").data[1].update(line_color='black')

fig = px.line(focredf.groupby(['area',pd.Grouper(key='closing_date',freq=bin_freq)]).mean().price_sold.reset_index(),x='closing_date',y='price_sold',color='area')
fig.add_traces(trendline)
fig.update_layout(title="Orange County Houses: Selling Price Last 3 Years (OCRE data)")
fig.update_xaxes(title_text="Year")
fig.update_yaxes(title_text="Price Sold")
fig.add_annotation(xref="paper",yref="paper",x=0.5,y=0,showarrow=False,text=f"filter: {filter}  ;  time binning: {bin_freq}")
fig.show()

* Within the price range of 750k to 2M, **the average price across all areas here has increase from 1.35M (Aug 2019) to 1.7M (Jan 2021)**
* This is easier to see by deselecting all areas in the legend (double click a single one to select only that area; then single click to deselect that area as well)

In [52]:
#hide_input
filter = 'price_list < 2e6 and price_list > 7.5e5 and closing_date > 20190701'
bin_freq = '2W'
focredf = ocredf.query(filter)

trendline = px.scatter(focredf.groupby([pd.Grouper(key='closing_date',freq=bin_freq)]).mean().price_sold_list_ratio.reset_index(),x='closing_date',y='price_sold_list_ratio',trendline="lowess").data[1].update(line_color='black')

fig = px.line(focredf.groupby(['area',pd.Grouper(key='closing_date',freq=bin_freq)]).mean().price_sold_list_ratio.reset_index(),x='closing_date',y='price_sold_list_ratio',color='area')
fig.add_traces(trendline)
fig.update_layout(title="Orange County Houses: How much do they sell over/under list price?")
fig.update_xaxes(title_text="Year")
fig.update_yaxes(title_text="Sold/List Price (%)")
fig.add_annotation(xref="paper",yref="paper",x=0.5,y=0,showarrow=False,text=f"filter: {filter}")
fig.show()

* The average selling price went from **97% of list (Jul 2019) to 104% of list (Jan 2022)** 
* This is easier to see by deselecting all areas in the legend (double click a single one to select only that area; then single click to deselect that area as well)

## CA Monthly Median Owner (Mortgage) Cost 

What are the typical costs that people in OC pay for their mortgage? We can use data from the [American Community Survey (search median mortgage)](https://data.census.gov/cedsci/) to look at this, though it only goes up to 2019.

In [53]:
#hide
# Read in owner cost data
sheetId = '1az-gajkCWX4GwgJnNMFX8gfeRvhRxAY3QMTowj0YTwE'
ccdf =  pd.read_csv(f'https://docs.google.com/spreadsheets/d/{sheetId}/export?gid=1172779957&format=csv',
                 skiprows=2,dtype='str')
ccdf.columns=['year','mmoc_total','mmoc_total_err','mmoc_mortg','mmoc_mortg_err','mmoc_nomortg','mmoc_nomortg_err','geoid','geo_area_name']
ccdf['zip'] = ccdf.geo_area_name.str.replace('ZCTA5 ','')
ccdf = ccdf.merge(azdf,left_on='zip',right_on='zip',how='inner')
# survey puts X,XXX+ when values exceed certain threshold (e.g., 4,000+  or 1,500+). Set these to the upper bound with 0.1 added so we can treat numerically but still differentiate later
print(ccdf.loc[ccdf.mmoc_mortg.str.contains('\+'),['year','mmoc_mortg','zip','area']])
for col in ccdf.columns[1:6]:
  ccdf.loc[ccdf[col].str.contains('\+'),col] = ccdf.loc[ccdf[col].str.contains('\+'),col].str.replace(',','').str.replace('\+','.1')
for col in ccdf.columns[1:6]:
  ccdf[col] = pd.to_numeric(ccdf[col].str.replace('[^0-9.]','',regex=True),errors='coerce')
ccdf.head()

     year mmoc_mortg    zip           area
5    2014     4,000+  92603         Irvine
7    2012     4,000+  92603         Irvine
8    2011     4,000+  92603         Irvine
108  2019     4,000+  92694   Ladera Ranch
109  2019     4,000+  92694  Mission Viejo
124  2011     4,000+  92694   Ladera Ranch
125  2011     4,000+  92694  Mission Viejo


Loading... (need help?)


In [54]:
#hide_input
# Make 
ccaydf = ccdf.groupby(['area','year']).median().mmoc_mortg.reset_index()
fig = px.scatter(ccaydf,x='year',y='mmoc_mortg',color='area',trendline='lowess')
fig.update_layout(title="Orange County Houses: Monthly Mortgage Cost Over Time")
fig.update_xaxes(title_text="Year")
fig.update_yaxes(title_text="Monthly Mortgage Cost (USD)")
fig.show()

* Note: Values that exceed $4,000 are reported in the data as "4,000+". They have been replaced with `4000.1` in the data above. This occurs for some areas of Irvine (92603 for 2011,2012,2014), Ladera Ranch/Mission Viejo (92694 for 2011, 2019). The estimates for these places/years may be much higher than shown in the plot.
* The takeaway is that mortgage costs are:
 * Ranked fairly stable across areas (high ranked areas remain high consistently)
 * Dropped to a low in 2015 and return to 2011 levels in 2019. Unfortunately, we can't yet see beyond 2019 with this data, though for 2021 onward, **we can impute a 40% increase in mortgage costs over 2019**.